## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
from pyspark.sql.functions import *

In [0]:
# Create a spark dataframe in the 

In [0]:
# File location and type
file_location = "/FileStore/tables/computerlogs/CpuLogData2019*.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df = df.select("DateTime","keyboard","mouse","user_name")\
        .withColumn('Date', split(df['DateTime'], ' ').getItem(0))\
        .withColumn('Time', split(df['DateTime'], ' ').getItem(1))\
        .withColumn('DateTime', to_timestamp(df['DateTime']))
print(df.count())

4122

In [0]:
storage_account_name = "saweekeightsparkstorage"

# Azure Storage Account Key
storage_account_key = "Your Storage Account key that you will find in access keys in your Storage Account "

# Azure Storage Account Source Container
container = "Your container name"

# Set the configuration details to read/write
spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

In [0]:
#Mount Azure Storage Account in your Databricks 
dbutils.fs.mount(
   source = "wasbs://{0}@{1}.blob.core.windows.net".format(container, storage_account_name),
   mount_point = "/mnt/sparklogfile",
   extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name): storage_account_key}
  )

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3348328071542889> in <module> 
 2 source = "wasbs://{0}@{1}.blob.core.windows.net" . format ( container , storage_account_name ) , 
 3 mount_point = "/mnt/sparklogfile" , 
 ----> 4 extra_configs = { "fs.azure.account.key.{0}.blob.core.windows.net" . format ( storage_account_name ) : storage_account_key } 
 5 )

 /local_disk0/tmp/1614143901425-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 322 exc . __context__ = None 
 323 exc . __cause__ = None 
 --> 324 raise exc
 325 return f_with_exception_handling
 326 

 ExecutionError : An error occurred while calling o257.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/sparklogfile; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/sparklogfile
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:470)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/sparklogfile
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:404)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:670)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:504)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:659)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:412)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:89)
	at com.databricks.backend.daemon.data.server.handler.CEMountHandler.receive(MountHandler.scala:141)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:96)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:96)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:288)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:248)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:53)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:49)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:432)
	at co

In [0]:
# Create a view or table

temp_table_name = "CpuLogData"

df.createOrReplaceTempView(temp_table_name)

In [0]:
highest_average_active_user_df = sqlContext.sql("select user_name,from_unixtime(ROUND(((count(*)*5)*60)/6,2),'HH:mm') as average_active_time_in_hours from `CpuLogData` where ( keyboard != 0 or mouse != 0 ) group by user_name order by average_active_time_in_hours desc")
display(highest_average_active_user_df)
highest_average_active_user_df.write.option("header",True) \
 .csv("/mnt/sparklogfile/highest_average_active_user_df.csv")

user_name,average_active_time_in_hours
deepshukla292@gmail.com,06:35
iamnzm@outlook.com,06:22
sharlawar77@gmail.com,06:20
salinabodale73@gmail.com,06:06
rahilstar11@gmail.com,05:32
markfernandes66@gmail.com,05:24
bhagyashrichalke21@gmail.com,05:00
damodharn21@gmail.com,02:39


In [0]:
lowest_average_active_user_df = sqlContext.sql("select user_name,from_unixtime(ROUND(((count(*)*5)*60)/6,2),'HH:mm')  as average_active_time_in_hours from `CpuLogData` where (keyboard != 0 or mouse != 0 ) group by user_name order by average_active_time_in_hours limit 1")
display(lowest_average_active_user_df)
lowest_average_active_user_df.write.option("header",True) \
 .csv("/mnt/sparklogfile/lowest_average_active_user_df.csv")

user_name,average_active_time_in_hours
damodharn21@gmail.com,02:39


In [0]:
# Highest Average Idle hour 
highest_average_idle_user_df = sqlContext.sql("select user_name,from_unixtime(ROUND(((count(*)*5)*60)/6,2),'HH:mm') as average_idle_time_in_hours from `CpuLogData` where (keyboard == 0 and mouse == 0 ) group by user_name order by average_idle_time_in_hours desc limit 1 ")
display(highest_average_idle_user_df)
highest_average_idle_user_df.write.option("header",True) \
 .csv("/mnt/sparklogfile/highest_average_idle_user_df.csv")

user_name,average_idle_time_in_hours
iamnzm@outlook.com,02:09


In [0]:

daily_session_start_time_df = sqlContext.sql("select Date,to_timestamp(min(DateTime)) as DateTime from CpuLogData group by Date order by Date")
temp_table_name = "view_session_start_time"
daily_session_start_time_df.createOrReplaceTempView(temp_table_name)
display(daily_session_start_time_df)


Date,DateTime
2019-09-16,2019-09-16T12:55:01.000+0000
2019-09-17,2019-09-17T08:25:01.000+0000
2019-09-18,2019-09-18T08:30:01.000+0000
2019-09-19,2019-09-19T08:40:02.000+0000
2019-09-20,2019-09-20T09:05:01.000+0000
2019-09-21,2019-09-21T09:10:01.000+0000


In [0]:
daily_login_time_df = sqlContext.sql("select user_name,Date,min(DateTime) as Datetime from `CpuLogData` where (keyboard != 0 or mouse != 0) group by user_name,Date order by Date")
temp_table_name = "view_daily_login_time"
daily_login_time_df.createOrReplaceTempView(temp_table_name)
display(daily_login_time_df)


user_name,Date,Datetime
deepshukla292@gmail.com,2019-09-16,2019-09-16T13:00:01.000+0000
salinabodale73@gmail.com,2019-09-16,2019-09-16T12:55:02.000+0000
rahilstar11@gmail.com,2019-09-16,2019-09-16T13:00:03.000+0000
iamnzm@outlook.com,2019-09-16,2019-09-16T13:00:01.000+0000
bhagyashrichalke21@gmail.com,2019-09-16,2019-09-16T12:55:01.000+0000
sharlawar77@gmail.com,2019-09-16,2019-09-16T13:00:04.000+0000
markfernandes66@gmail.com,2019-09-17,2019-09-17T10:50:01.000+0000
deepshukla292@gmail.com,2019-09-17,2019-09-17T09:30:01.000+0000
sharlawar77@gmail.com,2019-09-17,2019-09-17T10:45:02.000+0000
salinabodale73@gmail.com,2019-09-17,2019-09-17T10:15:01.000+0000


In [0]:
late_users_df = sqlContext.sql("select lut.user_name,\
Concat(cast(Round((Sum(lut.TimeDifference)/60)/6,2) AS INT),' Hour , ',Floor(Round((((Sum(lut.TimeDifference)/60)/6)%1)*60)),' Min') as average_late_time_in_hours, count(*) as daily_late_count from (Select lt.user_name,(unix_timestamp(lt.DateTime)-unix_timestamp(st.DateTime))/(60) as TimeDifference from view_daily_login_time as lt,view_session_start_time as st where lt.Date == st.Date) as lut where lut.TimeDifference != 0 group by lut.user_name order by average_late_time_in_hours desc")
display(late_users_df)
late_users_df.write.option("header",True) \
 .csv("/mnt/sparklogfile/late_users_df.csv")

user_name,average_late_time_in_hours,daily_late_count
markfernandes66@gmail.com,"1 Hour , 9 Min",5
sharlawar77@gmail.com,"1 Hour , 8 Min",5
salinabodale73@gmail.com,"1 Hour , 27 Min",6
bhagyashrichalke21@gmail.com,"1 Hour , 27 Min",5
rahilstar11@gmail.com,"1 Hour , 23 Min",6
damodharn21@gmail.com,"0 Hour , 59 Min",3
iamnzm@outlook.com,"0 Hour , 34 Min",4
deepshukla292@gmail.com,"0 Hour , 22 Min",4


--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3916484300341808> in <module> 
 3 display ( late_users_df ) 
 4 late_users_df . write . option ( "header" , True ) \ 
 ----> 5 . csv ( "/mnt/sparklogfile/late_users_df.csv" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in csv (self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encoding, emptyValue, lineSep) 
 1028 charToEscapeQuoteEscaping = charToEscapeQuoteEscaping , 
 1029 encoding=encoding, emptyValue=emptyValue, lineSep=lineSep)
 -> 1030 self . _jwrite . csv ( path ) 
 1031 
 1032 @ since ( 1.5 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : path dbfs:/mnt/sparklogfile/late_users_df.csv already exists.;